<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/fi_whisper_tiny_test_(edit).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-1x9c34l1
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-1x9c34l1
  Resolved https://github.com/huggingface/transformers to commit d269c4b2d7bbe8f25f3daab818bb13bb5ea4ca45
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.37.0.dev0-py3-none-any.whl size=8269300 sha256=c0f96a679da452851b2fe423c7fbb29604dd0dec0b9457b5d95b207adbc75f9b
  Stored in directory: /tmp/pip-ephem-wheel-cache-2wf4qzrd/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [3]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm

In [59]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "fi", split="test")
#column_name = "audio, normalized_text"

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [60]:
voxpopuli_data

Dataset({
    features: ['audio_id', 'language', 'audio', 'raw_text', 'normalized_text', 'gender', 'speaker_id', 'is_gold_transcript', 'accent'],
    num_rows: 478
})

In [61]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("base")

In [62]:
model = whisper.load_model("tiny")

In [63]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [64]:
Gender=voxpopuli_data['gender']
Audio=voxpopuli_data['audio']
text=voxpopuli_data['normalized_text']
Gender=pd.DataFrame(Gender)
Audio=pd.DataFrame(Audio)
text=pd.DataFrame(text)

In [65]:
text

,0
0,kiitän budjettivaliokuntaa ja sen varjoesittel...
1,syynä on myös puutteellinen tiedotus.
2,arvoisa puhemies indonesian valtiollinen ideol...
3,euroopan investointipankki sai pääosin sen mit...
4,ulkoasiainhallinto voi ja sen tulee pyrkiä ets...
...,...
473,meidän on myös kehitettävä vaihtoehtoisia mate...
474,suurin osa seminaariin osallistujista oli kuit...
475,jos ympäristövaliokunnan tahto menisi läpi eur...
476,haluan kiittää varjoesittelijöitä rakentavasta...


In [66]:
Gender.columns=['Gender']
text.columns=['Transcript']

In [50]:
file=pd.concat([Gender, text, Audio],axis=1)
file

,Gender,Transcript,path,array,sampling_rate
0,male,en daar die eigen middelen het ook mogelijk ma...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00738525390625, 0.00439453125, 0.0005798339...",16000
1,female,je hebt een zeer goede en sterke europese gedr...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000213623046875, 0.000152587890625, -0.0001...",16000
2,male,tegen die achtergrond vindt het voornemen dat ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.02880859375, -0.034271240234375, -0.033355...",16000
3,male,er is wel een duidelijke europese uitdaging al...,/root/.cache/huggingface/datasets/downloads/ex...,"[6.103515625e-05, 0.000152587890625, -6.103515...",16000
4,male,nog dramatischer is de situatie in libanon. da...,/root/.cache/huggingface/datasets/downloads/ex...,"[-3.0517578125e-05, -0.0001220703125, -0.00018...",16000
...,...,...,...,...,...
1132,male,de lijst van recente misdaden tegen christenen...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00103759765625, 0.00103759765625, 0.0012207...",16000
1133,male,ik wil graag dat de boeren op hun eigen benen ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.193695068359375, -0.7314453125, -0.4067687...",16000
1134,male,extra regels zullen het geschonden consumenten...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0008544921875, -0.00079345703125, -0.00012...",16000
1135,male,ik vind ook dat de wetenschap zoveel mogelijk ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001861572265625, 0.00115966796875, 0.00265...",16000


In [67]:
male_samples=file.loc[file['Gender'] == 'male']
male_samples

,Gender,Transcript,path,array,sampling_rate
0,male,en daar die eigen middelen het ook mogelijk ma...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00738525390625, 0.00439453125, 0.0005798339...",16000
2,male,tegen die achtergrond vindt het voornemen dat ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.02880859375, -0.034271240234375, -0.033355...",16000
3,male,er is wel een duidelijke europese uitdaging al...,/root/.cache/huggingface/datasets/downloads/ex...,"[6.103515625e-05, 0.000152587890625, -6.103515...",16000
4,male,nog dramatischer is de situatie in libanon. da...,/root/.cache/huggingface/datasets/downloads/ex...,"[-3.0517578125e-05, -0.0001220703125, -0.00018...",16000
5,male,wij als parlement moeten ook niet doen alsof w...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.010894775390625, 0.0115966796875, 0.0105895...",16000
...,...,...,...,...,...
1132,male,de lijst van recente misdaden tegen christenen...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00103759765625, 0.00103759765625, 0.0012207...",16000
1133,male,ik wil graag dat de boeren op hun eigen benen ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.193695068359375, -0.7314453125, -0.4067687...",16000
1134,male,extra regels zullen het geschonden consumenten...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0008544921875, -0.00079345703125, -0.00012...",16000
1135,male,ik vind ook dat de wetenschap zoveel mogelijk ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001861572265625, 0.00115966796875, 0.00265...",16000


In [68]:
female_samples=file.loc[file['Gender'] == 'female']
female_samples

,Gender,Transcript,path,array,sampling_rate
1,female,je hebt een zeer goede en sterke europese gedr...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000213623046875, 0.000152587890625, -0.0001...",16000
9,female,ik denk dat het een oogopener moet zijn of ik ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.076171875, -0.060577392578125, -0.03576660...",16000
10,female,deze soort kan tot dertig jaar oud worden.,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00701904296875, 0.002227783203125, -0.00909...",16000
13,female,voorzitter of je nu voor kernernergie bent of ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.010009765625, 0.00469970703125, 0.005584716...",16000
14,female,willen we dat mensen tweederangsburgers zijn v...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0162353515625, 0.031524658203125, 0.0357360...",16000
...,...,...,...,...,...
1125,female,dat kan natuurlijk.,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000244140625, -0.0001220703125, -6.1035156...",16000
1126,female,laten we elkaar vooral aan deze gemaakte afspr...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0003662109375, 0.000457763671875, 0.000701...",16000
1128,female,je ziet in veel landen dat er een quotasysteem...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.039276123046875, -0.052520751953125, -0.07...",16000
1129,female,ja dit parlement debatteert met emotie en nee ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.01861572265625, -0.033599853515625, -0.048...",16000


In [69]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="fi", fp16=False)
    return result["text"]

In [70]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [71]:
from jiwer import wer
from collections import Counter

def calculate_precision_recall_f1(ground_truth, hypothesis):
    ground_truth_words = set(ground_truth.split())
    hypothesis_words = set(hypothesis.split())

    true_positives = ground_truth_words.intersection(hypothesis_words)
    false_positives = hypothesis_words - ground_truth_words
    false_negatives = ground_truth_words - hypothesis_words

    precision = len(true_positives) / (len(true_positives) + len(false_positives)) if true_positives or false_positives else 0
    recall = len(true_positives) / (len(true_positives) + len(false_negatives)) if true_positives or false_negatives else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall else 0

    return precision, recall, f1_score

In [72]:

total_wer = 0
total_precision = 0
total_recall = 0
total_f1 = 0
num_sample = 0

for index,row in male_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)
    precision, recall, f1_score = calculate_precision_recall_f1(ground_truth_text, transcription)

    total_wer += sample_wer
    total_precision += precision
    total_recall += recall
    total_f1 += f1_score
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}, Precision: {precision}, Recall: {recall}, F1: {f1_score}")

# Calculate average metrics
average_wer = total_wer / num_sample
average_precision = total_precision / num_sample
average_recall = total_recall / num_sample
average_f1 = total_f1 / num_sample

print(f"Average WER: {average_wer}")
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1-Score: {average_f1}")

Processed 1 files, WER: 1.0, Precision: 0.0, Recall: 0.0, F1: 0
Processed 2 files, WER: 1.0, Precision: 0.0, Recall: 0.0, F1: 0
Processed 3 files, WER: 1.0, Precision: 0.0, Recall: 0.0, F1: 0
Processed 4 files, WER: 1.0, Precision: 0.0, Recall: 0.0, F1: 0
Processed 5 files, WER: 1.0, Precision: 0.0, Recall: 0.0, F1: 0
Processed 6 files, WER: 0.7631578947368421, Precision: 0.32, Recall: 0.23529411764705882, F1: 0.2711864406779661
Processed 7 files, WER: 1.3333333333333333, Precision: 0.0, Recall: 0.0, F1: 0
Processed 8 files, WER: 1.0, Precision: 0.0, Recall: 0.0, F1: 0
Processed 9 files, WER: 1.0, Precision: 0.0, Recall: 0.0, F1: 0
Processed 10 files, WER: 1.0, Precision: 0.0, Recall: 0.0, F1: 0
Processed 11 files, WER: 0.9777777777777777, Precision: 0.0625, Recall: 0.029411764705882353, F1: 0.039999999999999994
Processed 12 files, WER: 1.0, Precision: 0.0, Recall: 0.0, F1: 0
Processed 13 files, WER: 1.0, Precision: 0.0, Recall: 0.0, F1: 0
Processed 14 files, WER: 1.0, Precision: 0.0, 

In [73]:
total_wer = 0
total_precision = 0
total_recall = 0
total_f1 = 0
num_sample = 0

for index,row in female_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)
    precision, recall, f1_score = calculate_precision_recall_f1(ground_truth_text, transcription)


    total_wer += sample_wer
    total_precision += precision
    total_recall += recall
    total_f1 += f1_score
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}, Precision: {precision}, Recall: {recall}, F1: {f1_score}")

# Calculate average metrics
average_wer = total_wer / num_sample
average_precision = total_precision / num_sample
average_recall = total_recall / num_sample
average_f1 = total_f1 / num_sample

print(f"Average WER: {average_wer}")
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1-Score: {average_f1}")

Processed 1 files, WER: 1.0, Precision: 0.0, Recall: 0.0, F1: 0
Processed 2 files, WER: 0.975609756097561, Precision: 0.03571428571428571, Recall: 0.030303030303030304, F1: 0.03278688524590164
Processed 3 files, WER: 1.0, Precision: 0.0, Recall: 0.0, F1: 0
Processed 4 files, WER: 1.0, Precision: 0.0, Recall: 0.0, F1: 0
Processed 5 files, WER: 1.0, Precision: 0.0, Recall: 0.0, F1: 0
Processed 6 files, WER: 1.0, Precision: 0.0, Recall: 0.0, F1: 0
Processed 7 files, WER: 1.0833333333333333, Precision: 0.0, Recall: 0.0, F1: 0
Processed 8 files, WER: 1.0, Precision: 0.0, Recall: 0.0, F1: 0
Processed 9 files, WER: 1.5, Precision: 0.0, Recall: 0.0, F1: 0
Processed 10 files, WER: 1.4333333333333333, Precision: 0.0, Recall: 0.0, F1: 0
Processed 11 files, WER: 1.0, Precision: 0.0, Recall: 0.0, F1: 0
Processed 12 files, WER: 1.0, Precision: 0.0, Recall: 0.0, F1: 0
Processed 13 files, WER: 1.0, Precision: 0.0, Recall: 0.0, F1: 0
Processed 14 files, WER: 2.04, Precision: 0.02040816326530612, Recall